### Json Parsing And Processing

### Create a sample JSON file

In [ ]:
import json
import os
os.makedirs('data/json_files', exist_ok=True)

In [2]:
json_data = {
    "company": "TechCorp",
    "employees": [
        {
            "id": 1,
            "name": "John Doe",
            "role": "Software Engineer",
            "skills": ["Python", "JavaScript", "React"],
            "projects": [
                {"name": "RAG System", "status": "In Progress"},
                {"name": "Data Pipeline", "status": "Completed"}
            ]
        },
        {
            "id": 2,
            "name": "Jane Smith",
            "role": "Data Scientist",
            "skills": ["Python", "Machine Learning", "SQL"],
            "projects": [
                {"name": "ML Model", "status": "In Progress"},
                {"name": "Analytics Dashboard", "status": "Planning"}
            ]
        }
    ],
    "departments": {
        "engineering": {
            "head": "Mike Johnson",
            "budget": 1000000,
            "team_size": 25
        },
        "data_science": {
            "head": "Sarah Williams",
            "budget": 750000,
            "team_size": 15
        }
    }
}

In [3]:
with open('data/json_files/company_data.json', 'w') as f:
    json.dump(json_data, f, indent=2)

### Create a sample JSONL file

In [5]:
jsonl_data = [
    {"timestamp": "2024-01-01", "event": "user_login", "user_id": 123},
    {"timestamp": "2024-01-01", "event": "page_view", "user_id": 123, "page": "/home"},
    {"timestamp": "2024-01-01", "event": "purchase", "user_id": 123, "amount": 99.99}
]

with open('data/json_files/events.jsonl', 'w') as f:
    for item in jsonl_data:
        f.write(json.dumps(item) + '\n')

### JSONLoader

In [9]:
from langchain_community.document_loaders import JSONLoader
import json

loader = JSONLoader(
    file_path='data/json_files/company_data.json', 
    jq_schema='.employees[]', 
    text_content=False)
docs = loader.load()
print(f"Loaded {len(docs)} documents")
print(f"First employee: {docs[0].page_content}")

Loaded 2 documents
First employee: {"id": 1, "name": "John Doe", "role": "Software Engineer", "skills": ["Python", "JavaScript", "React"], "projects": [{"name": "RAG System", "status": "In Progress"}, {"name": "Data Pipeline", "status": "Completed"}]}


In [8]:
docs

[Document(metadata={'source': '/Users/apple/Desktop/AI Practice Projects/RAG_Bootcamp/0-DataIngestParsing/data/json_files/company_data.json', 'seq_num': 1}, page_content='{"id": 1, "name": "John Doe", "role": "Software Engineer", "skills": ["Python", "JavaScript", "React"], "projects": [{"name": "RAG System", "status": "In Progress"}, {"name": "Data Pipeline", "status": "Completed"}]}'),
 Document(metadata={'source': '/Users/apple/Desktop/AI Practice Projects/RAG_Bootcamp/0-DataIngestParsing/data/json_files/company_data.json', 'seq_num': 2}, page_content='{"id": 2, "name": "Jane Smith", "role": "Data Scientist", "skills": ["Python", "Machine Learning", "SQL"], "projects": [{"name": "ML Model", "status": "In Progress"}, {"name": "Analytics Dashboard", "status": "Planning"}]}')]

### Custom Json Processing

In [11]:
from typing import List
from langchain_core.documents import Document


def process_json(filepath):
    with open(filepath, 'r') as f:
        data = json.load(f)

    documents = []

    for emp in data.get('employees', []):
        content = f"""Employee Profile:
        Name: {emp['name']}
        Role: {emp['role']}
        Skills: {', '.join(emp['skills'])}

        Projects:"""
        for project in emp.get('projects', []):
            content += f"\n- {project['name']} (Status: {project['status']})"

        doc = Document(
            page_content=content,
            metadata={
                'source': filepath,
                'data_type': 'employee_profile',
                'employee_id': emp['id'],
                'employee_name': emp['name'],
                'role': emp['role']
            }
        )
        documents.append(doc)
    return documents
        

In [12]:
process_json('data/json_files/company_data.json')

[Document(metadata={'source': 'data/json_files/company_data.json', 'data_type': 'employee_profile', 'employee_id': 1, 'employee_name': 'John Doe', 'role': 'Software Engineer'}, page_content='Employee Profile:\n        Name: John Doe\n        Role: Software Engineer\n        Skills: Python, JavaScript, React\n\n        Projects:\n- RAG System (Status: In Progress)\n- Data Pipeline (Status: Completed)'),
 Document(metadata={'source': 'data/json_files/company_data.json', 'data_type': 'employee_profile', 'employee_id': 2, 'employee_name': 'Jane Smith', 'role': 'Data Scientist'}, page_content='Employee Profile:\n        Name: Jane Smith\n        Role: Data Scientist\n        Skills: Python, Machine Learning, SQL\n\n        Projects:\n- ML Model (Status: In Progress)\n- Analytics Dashboard (Status: Planning)')]